<a href="https://colab.research.google.com/github/SheshanthV04/FreeCodeCamp-Python-ML/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data = pd.read_csv(train_file_path, sep="\t", header=None, names=["label", "message"])
test_data = pd.read_csv(test_file_path, sep="\t", header=None, names=["label", "message"])

train_data.head()


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_data['message'])  # messages → numbers
X_test = vectorizer.transform(test_data['message'])        # same for test data

encoder = LabelEncoder()
y_train = encoder.fit_transform(train_data['label'])       # "ham" → 0, "spam" → 1
y_test = encoder.transform(test_data['label'])

print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)


In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()

model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
print(f"Model accuracy on test set: {accuracy:.2f}")


In [ ]:
# Function to predict messages based on the trained model
def predict_message(pred_text):
    message_vector = vectorizer.transform([pred_text])

    prediction_prob = model.predict_proba(message_vector)[0][1]

    prediction_label = model.predict(message_vector)[0]

    label_text = encoder.inverse_transform([prediction_label])[0]

    return [prediction_prob, label_text]


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
